--- Day 9: Rope Bridge ---

This rope bridge creaks as you walk along it. You aren't sure how old it is, or whether it can even support your weight.

It seems to support the Elves just fine, though. The bridge spans a gorge which was carved out by the massive river far below you.

You step carefully; as you do, the ropes stretch and twist. You decide to distract yourself by modeling rope physics; maybe you can even figure out where **not** to step.

Consider a rope with a knot at each end; these knots mark the **head** and the **tail** of the rope. If the head moves far enough away from the tail, the tail is pulled toward the head.

Due to nebulous reasoning involving [Planck lengths](https://en.wikipedia.org/wiki/Planck_units#Planck_length), you should be able to model the positions of the knots on a two-dimensional grid. Then, by following a hypothetical **series of motions** (your puzzle input) for the head, you can determine how the tail will move.

Due to the aforementioned Planck lengths, the rope must be quite short; in fact, the head (`H`) and tail (`T`) must **always be touching** (diagonally adjacent and even overlapping both count as touching):
```
....
.TH.
....

....
.H..
..T.
....

...
.H. (H covers T)
...
```
If the head is ever two steps directly up, down, left, or right from the tail, the tail must also move one step in that direction so it remains close enough:
```
.....    .....    .....
.TH.. -> .T.H. -> ..TH.
.....    .....    .....

...    ...    ...
.T.    .T.    ...
.H. -> ... -> .T.
...    .H.    .H.
...    ...    ...
```
Otherwise, if the head and tail aren't touching and aren't in the same row or column, the tail always moves one step diagonally to keep up:
```
.....    .....    .....
.....    ..H..    ..H..
..H.. -> ..... -> ..T..
.T...    .T...    .....
.....    .....    .....

.....    .....    .....
.....    .....    .....
..H.. -> ...H. -> ..TH.
.T...    .T...    .....
.....    .....    .....
```
You just need to work out where the tail goes as the head follows a series of motions. Assume the head and the tail both start at the same position, overlapping.

For example:
```
R 4
U 4
L 3
D 1
R 4
D 1
L 5
R 2
```
This series of motions moves the head **right** four steps, then **up** four steps, then **left** three steps, then **down** one step, and so on. After each step, you'll need to update the position of the tail if the step means the head is no longer adjacent to the tail. Visually, these motions occur as follows (s marks the starting position as a reference point):

== Initial State ==
```
......
......
......
......
H.....  (H covers T, s)

== R 4 ==

......
......
......
......
TH....  (T covers s)

......
......
......
......
sTH...

......
......
......
......
s.TH..

......
......
......
......
s..TH.

== U 4 ==

......
......
......
....H.
s..T..

......
......
....H.
....T.
s.....

......
....H.
....T.
......
s.....

....H.
....T.
......
......
s.....

== L 3 ==

...H..
....T.
......
......
s.....

..HT..
......
......
......
s.....

.HT...
......
......
......
s.....

== D 1 ==

..T...
.H....
......
......
s.....

== R 4 ==

..T...
..H...
......
......
s.....

..T...
...H..
......
......
s.....

......
...TH.
......
......
s.....

......
....TH
......
......
s.....

== D 1 ==

......
....T.
.....H
......
s.....

== L 5 ==

......
....T.
....H.
......
s.....

......
....T.
...H..
......
s.....

......
......
..HT..
......
s.....

......
......
.HT...
......
s.....

......
......
HT....
......
s.....

== R 2 ==

......
......
.H....  (H covers T)
......
s.....

......
......
.TH...
......
s.....
```
After simulating the rope, you can count up all of the positions the **tail visited at least once**. In this diagram, `s` again marks the starting position (which the tail also visited) and `#` marks other positions the tail visited:
```
..##..
...##.
.####.
....#.
s###..
```
So, there are `13` positions the tail visited at least once.

Simulate your complete hypothetical series of motions. **How many positions does the tail of the rope visit at least once?**

In [1]:
# Data Load
example_lines = [
    "R 4",
    "U 4",
    "L 3",
    "D 1",
    "R 4",
    "D 1",
    "L 5",
    "R 2",
]

example_lines_2 = [
    "R 5",
    "U 8",
    "L 8",
    "D 3",
    "R 17",
    "D 10",
    "L 25",
    "U 20",
]

with open("day_9_input.txt") as _file:
    data_lines = _file.readlines()

In [2]:
# Common Code
from __future__ import annotations
import typing
from collections import defaultdict


class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __members(self):
        return (self.x, self.y)

    def __repr__(self):
        return str(self.__members())

    def __add__(self, other):
        return Vector(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return Vector(self.x - other.x, self.y - other.y)

    def __eq__(self, other):
        return self.__members() == other.__members()

    def __hash__(self):
        return hash(self.__members())

    @classmethod
    def copy(cls, other):
        return cls(other.x, other.y)

    @classmethod
    def zero(cls):
        return cls(0, 0)

    @classmethod
    def one(cls):
        return cls(1, 1)


class Marker:
    mark: Vector

    def __init__(self):
        self.counts = defaultdict(lambda: 0)
        self._mark = None
        self.mark = Vector.zero()

    @property
    def mark(self):
        return Vector.copy(self._mark)

    @mark.setter
    def mark(self, value):
        self._mark = Vector.copy(value)
        self.counts[value] += 1


class Direction(Vector):
    _DIRECTIONS = {
        "R": (1, 0),
        "L": (-1, 0),
        "U": (0, -1),
        "D": (0, 1)
    }

    @classmethod
    def parse(cls, symbol):
        if symbol in cls._DIRECTIONS:
            return cls(*cls._DIRECTIONS[symbol])
        raise RuntimeError(f"Unexpected symbol: {symbol}")


class Move:
    def __init__(self, direction: Direction, quantity):
        self.direction = direction
        self.quantity = quantity

    @classmethod
    def parse(cls, line):
        symbol, quantity = line.split()
        return cls(Direction.parse(symbol), int(quantity))


class Rope:
    _locations: dict[Vector, int]

    def __init__(self, moves: typing.Iterable[Move], knots):
        self._moves = moves

        self._knots = [Marker() for _ in range(knots)]

        self._top_left = Vector.zero()
        self._bot_right = Vector.zero()

    @classmethod
    def parse(cls, lines, knots=2):
        return cls([Move.parse(line.strip()) for line in lines], knots)

    def _apply_single_move(self, move: Direction):
        knot = self._knots[0]
        position = knot.mark
        knot.mark += move

        self._top_left.x = min(knot.mark.x, self._top_left.x)
        self._top_left.y = min(knot.mark.y, self._top_left.y)
        self._bot_right.x = max(knot.mark.x, self._bot_right.x)
        self._bot_right.y = max(knot.mark.y, self._bot_right.y)

        def get_primary_offset_direction(diff):
            direction = Vector.zero()

            if diff.x <= -2:
                direction.x = -1
            elif diff.x >= 2:
                direction.x = 1

            if diff.y <= -2:
                direction.y = -1
            elif diff.y >= 2:
                direction.y = 1

            return direction

        for new_knot in self._knots[1:]:
            diff = new_knot.mark - knot.mark
            manhattan_distance = abs(diff.x) + abs(diff.y)

            if manhattan_distance >= 3 or abs(diff.x) >= 2 or abs(diff.y) >= 2:
                offset = get_primary_offset_direction(diff)
                new_knot.mark = knot.mark + offset
            else:
                break

            knot = new_knot

    def _apply_move(self, move: Move, display: bool):
        if display:
            print(f"== {move.direction} {move.quantity} ==")
            print()
        for _ in range(move.quantity):
            self._apply_single_move(move.direction)
            if display:
                self._print_state()

    def _get_state(self, show_tail_positions):
        dimensions = self._bot_right - self._top_left + Vector.one()
        board = ["." * dimensions.x for _ in range(dimensions.y)]

        zero_index = Vector.zero() - self._top_left

        def replace_char_at(pos: Vector, new_chr, force=False):
            pos += zero_index
            string = board[pos.y]
            if force or string[pos.x] == ".":
                board[pos.y] = string[:pos.x] + new_chr + string[pos.x + 1:]

        for i, next_knot in enumerate(self._knots):
            replace_char_at(next_knot.mark, "H" if i == 0 else str(i))

        if show_tail_positions:
            for position in self._knots[-1].counts.keys():
                replace_char_at(position, "#", force=True)

        replace_char_at(Vector.zero(), "s", force=True)

        return board

    def _print_state(self, show_tail_positions=False):
        state = self._get_state(show_tail_positions)

        for row in state:
            print(row)
        print()

    def apply_moves(self, display=False, display_after_round=False):
        if display or display_after_round:
            print("== INITIAL STATE ==")
            print()
            self._print_state()

        for move in self._moves:
            self._apply_move(move, display)
            if display_after_round:
                self._print_state()

    def get_tail_location_count(self):
        return len(self._knots[-1].counts)

    def print_tail_locations(self):
        self._print_state(show_tail_positions=True)

In [3]:
# Solution 1 Code
def solution_1(lines, display):
    rope = Rope.parse(lines)

    rope.apply_moves(display)

    if display:
        rope.print_tail_locations()

    return rope.get_tail_location_count()


def test_solution_1():
    actual = solution_1(example_lines, True)
    expected = 13

    print(actual, expected)
    assert actual == expected
test_solution_1()

== INITIAL STATE ==

s

== (1, 0) 4 ==

sH

s1H

s.1H

s..1H

== (0, -1) 4 ==

....H
s..1.

....H
....1
s....

....H
....1
.....
s....

....H
....1
.....
.....
s....

== (-1, 0) 3 ==

...H.
....1
.....
.....
s....

..H1.
.....
.....
.....
s....

.H1..
.....
.....
.....
s....

== (0, 1) 1 ==

..1..
.H...
.....
.....
s....

== (1, 0) 4 ==

..1..
..H..
.....
.....
s....

..1..
...H.
.....
.....
s....

.....
...1H
.....
.....
s....

......
....1H
......
......
s.....

== (0, 1) 1 ==

......
....1.
.....H
......
s.....

== (-1, 0) 5 ==

......
....1.
....H.
......
s.....

......
....1.
...H..
......
s.....

......
......
..H1..
......
s.....

......
......
.H1...
......
s.....

......
......
H1....
......
s.....

== (1, 0) 2 ==

......
......
.H....
......
s.....

......
......
.1H...
......
s.....

..##..
...##.
.####.
....#.
s###..

13 13


In [4]:
# Solution 1
solution_1(data_lines, False)

6090

--- Part Two ---

A rope snaps! Suddenly, the river is getting a lot closer than you remember. The bridge is still there, but some of the ropes that broke are now whipping toward you as you fall through the air!

The ropes are moving too quickly to grab; you only have a few seconds to choose how to arch your body to avoid being hit. Fortunately, your simulation can be extended to support longer ropes.

Rather than two knots, you now must simulate a rope consisting of **ten** knots. One knot is still the head of the rope and moves according to the series of motions. Each knot further down the rope follows the knot in front of it using the same rules as before.

Using the same series of motions as the above example, but with the knots marked H, 1, 2, ..., 9, the motions now occur as follows:
```
== Initial State ==

......
......
......
......
H.....  (H covers 1, 2, 3, 4, 5, 6, 7, 8, 9, s)

== R 4 ==

......
......
......
......
1H....  (1 covers 2, 3, 4, 5, 6, 7, 8, 9, s)

......
......
......
......
21H...  (2 covers 3, 4, 5, 6, 7, 8, 9, s)

......
......
......
......
321H..  (3 covers 4, 5, 6, 7, 8, 9, s)

......
......
......
......
4321H.  (4 covers 5, 6, 7, 8, 9, s)

== U 4 ==

......
......
......
....H.
4321..  (4 covers 5, 6, 7, 8, 9, s)

......
......
....H.
.4321.
5.....  (5 covers 6, 7, 8, 9, s)

......
....H.
....1.
.432..
5.....  (5 covers 6, 7, 8, 9, s)

....H.
....1.
..432.
.5....
6.....  (6 covers 7, 8, 9, s)

== L 3 ==

...H..
....1.
..432.
.5....
6.....  (6 covers 7, 8, 9, s)

..H1..
...2..
..43..
.5....
6.....  (6 covers 7, 8, 9, s)

.H1...
...2..
..43..
.5....
6.....  (6 covers 7, 8, 9, s)

== D 1 ==

..1...
.H.2..
..43..
.5....
6.....  (6 covers 7, 8, 9, s)

== R 4 ==

..1...
..H2..
..43..
.5....
6.....  (6 covers 7, 8, 9, s)

..1...
...H..  (H covers 2)
..43..
.5....
6.....  (6 covers 7, 8, 9, s)

......
...1H.  (1 covers 2)
..43..
.5....
6.....  (6 covers 7, 8, 9, s)

......
...21H
..43..
.5....
6.....  (6 covers 7, 8, 9, s)

== D 1 ==

......
...21.
..43.H
.5....
6.....  (6 covers 7, 8, 9, s)

== L 5 ==

......
...21.
..43H.
.5....
6.....  (6 covers 7, 8, 9, s)

......
...21.
..4H..  (H covers 3)
.5....
6.....  (6 covers 7, 8, 9, s)

......
...2..
..H1..  (H covers 4; 1 covers 3)
.5....
6.....  (6 covers 7, 8, 9, s)

......
...2..
.H13..  (1 covers 4)
.5....
6.....  (6 covers 7, 8, 9, s)

......
......
H123..  (2 covers 4)
.5....
6.....  (6 covers 7, 8, 9, s)

== R 2 ==

......
......
.H23..  (H covers 1; 2 covers 4)
.5....
6.....  (6 covers 7, 8, 9, s)

......
......
.1H3..  (H covers 2, 4)
.5....
6.....  (6 covers 7, 8, 9, s)
```
Now, you need to keep track of the positions the new tail, `9`, visits. In this example, the tail never moves, and so it only visits **`1`** position. However, **be careful**: more types of motion are possible than before, so you might want to visually compare your simulated rope to the one above.

Here's a larger example:
```
R 5
U 8
L 8
D 3
R 17
D 10
L 25
U 20
```
These motions occur as follows (individual steps are not shown):
```
== Initial State ==

..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
...........H..............  (H covers 1, 2, 3, 4, 5, 6, 7, 8, 9, s)
..........................
..........................
..........................
..........................
..........................

== R 5 ==

..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
...........54321H.........  (5 covers 6, 7, 8, 9, s)
..........................
..........................
..........................
..........................
..........................

== U 8 ==

..........................
..........................
..........................
..........................
..........................
..........................
..........................
................H.........
................1.........
................2.........
................3.........
...............54.........
..............6...........
.............7............
............8.............
...........9..............  (9 covers s)
..........................
..........................
..........................
..........................
..........................

== L 8 ==

..........................
..........................
..........................
..........................
..........................
..........................
..........................
........H1234.............
............5.............
............6.............
............7.............
............8.............
............9.............
..........................
..........................
...........s..............
..........................
..........................
..........................
..........................
..........................

== D 3 ==

..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
.........2345.............
........1...6.............
........H...7.............
............8.............
............9.............
..........................
..........................
...........s..............
..........................
..........................
..........................
..........................
..........................

== R 17 ==

..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
................987654321H
..........................
..........................
..........................
..........................
...........s..............
..........................
..........................
..........................
..........................
..........................

== D 10 ==

..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
...........s.........98765
.........................4
.........................3
.........................2
.........................1
.........................H

== L 25 ==

..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
...........s..............
..........................
..........................
..........................
..........................
H123456789................

== U 20 ==

H.........................
1.........................
2.........................
3.........................
4.........................
5.........................
6.........................
7.........................
8.........................
9.........................
..........................
..........................
..........................
..........................
..........................
...........s..............
..........................
..........................
..........................
..........................
..........................
```
Now, the tail (`9`) visits **`36`** positions (including `s`) at least once:
```
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
..........................
#.........................
#.............###.........
#............#...#........
.#..........#.....#.......
..#..........#.....#......
...#........#.......#.....
....#......s.........#....
.....#..............#.....
......#............#......
.......#..........#.......
........#........#........
.........########.........
```
Simulate your complete series of motions on a larger rope with ten knots. How many positions does the tail of the rope visit at least once?

In [5]:
# Solution 2 Code
def solution_2(lines, display):
    rope = Rope.parse(lines, 10)

    rope.apply_moves(False, display)

    if display:
        rope.print_tail_locations()

    return rope.get_tail_location_count()

def test_solution_2():
    actual = solution_2(example_lines, True)
    expected = 1

    print(actual, expected)
    assert actual == expected

    actual = solution_2(example_lines_2, True)
    expected = 36

    print(actual, expected)
    assert actual == expected
test_solution_2()

== INITIAL STATE ==

s

s321H

....H
....1
..432
.5...
s....

.H1..
...2.
..43.
.5...
s....

..1..
.H.2.
..43.
.5...
s....

......
...21H
..43..
.5....
s.....

......
...21.
..43.H
.5....
s.....

......
......
H123..
.5....
s.....

......
......
.1H3..
.5....
s.....

......
......
.1H3..
.5....
s.....

1 1
== INITIAL STATE ==

s

s4321H

.....H
.....1
.....2
.....3
....54
...6..
..7...
.8....
s.....

H1234....
....5....
....6....
....7....
....8....
....9....
.........
.........
...s.....

.........
.2345....
1...6....
H...7....
....8....
....9....
.........
.........
...s.....

..................
..................
..................
........987654321H
..................
..................
..................
..................
...s..............

..................
..................
..................
..................
..................
..................
..................
..................
...s.........98765
.................4
.................3
.................2
..............

In [6]:
# Solution 2
solution_2(data_lines, False)

2566